# Worksheet 07

Name: Jack Ruan \
UID: U46736284

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [9]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.713577130336239, 4.46688240411532, 5.443907052312175, 3.657457615745148, 5.272851142421775, 5.262119619736766, 4.704556084779029, 5.2494435196446645, 3.3994387479800463, 5.1465887076106505]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [10]:
mean = 8
stdev = 1

c2 = np.random.normal(mean, stdev, 10).tolist()
print(c2)

[8.759404997121012, 6.811189945345712, 8.833521940570712, 7.675094405805697, 8.450751069977734, 7.768127111567409, 7.543580666983614, 9.048501033084309, 6.6290631377939855, 7.290654324837128]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [11]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.1465887076106505, 3.3994387479800463, 7.290654324837128, 5.2494435196446645, 6.6290631377939855, 9.048501033084309, 7.543580666983614, 7.768127111567409, 4.704556084779029, 8.450751069977734]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean for c1 is 5 and std for c1 is 1. The mean for c2 is 8 and std for c2 is 1.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [12]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [len(c1) / (len(c1) + len(c2)), len(c2) / (len(c1) + len(c2))]
mean = [sum(c1)/len(c1), sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.290654324837128, 6.6290631377939855, 9.048501033084309, 7.543580666983614, 7.768127111567409, 8.450751069977734]
[5.1465887076106505, 3.3994387479800463, 5.2494435196446645, 4.704556084779029]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 7.788446224040697,  mean_2 = 4.625006765003597
var_1 = 0.6131206256163065,  var_2 = 0.5425785105070471


/Users/jackruan/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


They are pretty close to the true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [13]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.1465887076106505
probability of observing that point if it came from cluster 0 =  6.049629556666012e-05
probability of observing that point if it came from cluster 1 =  0.46321357029690957
point =  3.3994387479800463
probability of observing that point if it came from cluster 0 =  4.852082277283081e-12
probability of observing that point if it came from cluster 1 =  0.057350706189465896
point =  7.290654324837128
probability of observing that point if it came from cluster 0 =  0.4679776318699961
probability of observing that point if it came from cluster 1 =  4.2189159010006306e-06
point =  5.2494435196446645
probability of observing that point if it came from cluster 0 =  0.00012289754183136592
probability of observing that point if it came from cluster 1 =  0.37917186530557256
point =  6.6290631377939855
probability of observing that point if it came from cluster 0 =  0.10887018813895902
probability of observing that point if it came from cluster 1 =  0.0008016988899109971

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [14]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.5995794419107774,  P(C_2) = 0.40042055808922267
mean_1 = 7.789097080434833,  mean_2 = 4.627354721225401
var_1 = 0.6132121122851942,  var_2 = 0.5468540696496336


These values aren't that different. The variances are marginally closer to what they should be.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [15]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

5.1465887076106505
Probability of coming from C_1 = 0.00019447070171676797
Probability of coming from C_2 = 0.9998055292982833

3.3994387479800463
Probability of coming from C_1 = 1.2388272320863433e-10
Probability of coming from C_2 = 0.9999999998761172

7.290654324837128
Probability of coming from C_1 = 0.9999926319619137
Probability of coming from C_2 = 7.368038086369737e-06

5.2494435196446645
Probability of coming from C_1 = 0.00048058474848533235
Probability of coming from C_2 = 0.9995194152515147

6.6290631377939855
Probability of coming from C_1 = 0.9945090373114106
Probability of coming from C_2 = 0.005490962688589254

9.048501033084309
Probability of coming from C_1 = 0.9999999999999605
Probability of coming from C_2 = 3.9546848886001924e-14

7.543580666983614
Probability of coming from C_1 = 0.9999994580252579
Probability of coming from C_2 = 5.419747421578221e-07

7.768127111567409
Probability of coming from C_1 = 0.9999999484963854
Probability of coming from C_2 = 5.150361

There seem to be a lot more definitive decisions being made (probability of 1) which wasn't really happening with the kmeans implementation.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [16]:
labels = []

for i in range(len(data)):
    if prob_c0_x[i] > prob_c1_x[i]:
        labels.append(0)
    else:
        labels.append(1)

print(labels)

[1, 1, 0, 1, 0, 0, 0, 0, 1, 0]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.